In [139]:
#
# Emre Alca
# University of Pennsylvania
# Created on Sat Nov 22 2025
#

In [140]:
import numpy as np
import trimesh
import tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

%matplotlib widget

from src import spindle_state as ss


Need to implement:
basic functionality for short timescales:
- [x] spindle state
- [x] pulling forces of a given spindle state
- [x] pushing forces of a given spindle state
- [x] equations of motion
- [x] time evolution

Turnover regimes
- [ ] Markovian catastrophe and nucleation 
- [ ] gradient descent
    - [x] cost function
    - [x] sampling spatial nucleation distribution
    - [ ] sampling spatial catastrophe distribution
    - [ ] sampling length nucleation distribution
    - [ ] sampling length catastrophe distribution
    - [ ] spindle update
    - [ ] stochastic gradient descent loop



In [141]:
test_spindle_lattice = np.array([
    [1, 0, 0],
    [-1, 0, 0],
    [0, 1, 0],
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, -1],
])

expected_mt_vecs = np.array([
       [ 0.5,  0. ,  0. ],
       [-1.5,  0. ,  0. ],
       [-0.5,  1. ,  0. ],
       [-0.5, -1. ,  0. ],
       [-0.5,  0. ,  1. ],
       [-0.5,  0. , -1. ]])

test_spindle_state = np.array([1, 1, 3, 3, 1, 1])

test_spindle = ss.Spindle(np.array([0, 0, 0]), test_spindle_state, test_spindle_lattice)

In [142]:
# -- run and tumble approach to gradient descent -- 

# so long as the gradient is decreasing according to the usual time evolution, the spindle state is left unchanged

# if the usual time evolution leads to an increase in cost, we modify the spindle state according to our biased sampling distributions



# test_spindle.add_microtubules([0, 2, 5])

f_vec = np.zeros(len(test_spindle.spindle_state)) + (test_spindle.spindle_state==3).astype(int) - (test_spindle.spindle_state == 1).astype(int)

f_mhat_vec =  (test_spindle.mt_dirs.T * f_vec.T).T

print(f_mhat_vec)

minus_r_hat = -ss.normalize_vecs(test_spindle.mtoc_pos)[0]

# ((f_mhat_vec @ minus_r_hat + 1) / np.pi).T


select_empty_sites_only = np.zeros(len(test_spindle.spindle_state)) + (test_spindle.spindle_state==3).astype(int) + (test_spindle.spindle_state == 1).astype(int)

((f_mhat_vec @ minus_r_hat + 1) / np.pi) * select_empty_sites_only

[[-1. -0. -0.]
 [ 1. -0. -0.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]
 [-0. -0. -1.]
 [-0. -0.  1.]]


array([0.31830989, 0.31830989, 0.31830989, 0.31830989, 0.31830989,
       0.31830989])

In [ ]:
def biased_spatial_catastrophe_distribution(self):

        # find the unoccupied sites and set pulling = 1, pushing = -1 
        f_vec = np.zeros(len(self.spindle_state)) + (self.spindle_state==3).astype(int) - (self.spindle_state == 1).astype(int)
        f_mhat_vec =  (self.mt_dirs.T * f_vec.T).T

        # minus the norm of the MTOC position
        minus_r_hat = -normalize_vecs(self.mtoc_pos)[0]

        # normalized distribution biased by the dot product between each mt direction vector and minus_r_hat
        f_mhat_dot_minus_r_hat = f_mhat_vec @ minus_r_hat

        # transforming dot product into biased distributions
        biased_spatial_nucleation_distribution = (f_mhat_dot_minus_r_hat + 1) / np.pi

        select_empty_sites_only = np.zeros(len(self.spindle_state)) + (self.spindle_state==3).astype(int) + (self.spindle_state == 1).astype(int)

        return biased_spatial_nucleation_distribution * select_empty_sites_only